## 用决策树来分类贷款是否优良

[LendingClub](https://www.lendingclub.com/) 是一家贷款公司. 在本次作业中,我们需要手动实现决策树来预测一份贷款是否安全，并对比不同复杂度下决策树的表现

In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import tree
from sklearn import metrics

## 读取数据

In [2]:
#data = pd.read_csv(os.path.join("data", "loan_sub.csv"), sep=',')
data = pd.read_csv(os.path.join("./", "loan_sub.csv"), sep=',')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 打印可用特征

In [3]:
data.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'is_inc_v', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'not_compliant', 'status', 'inactive_loans', 'bad_loans',
       'emp_length_num', 'grade_num', 'sub_gra

## 预处理预测目标

预测目标是一列'bad_loans'的数据。其中**1**表示的是不良贷款，**0**表示的是优质贷款。

将预测目标处理成更符合直觉的标签，创建一列 `safe_loans`. 并且: 

* **+1** 表示优质贷款, 
* **-1** 表示不良贷款. 

In [4]:
# safe_loans =  1 => safe
# safe_loans = -1 => risky
data['safe_loans'] = data['bad_loans'].apply(lambda x : +1 if x==0 else -1)
data = data.drop('bad_loans', axis=1)

## 打印优质贷款与不良贷款的比例

In [5]:
data['safe_loans'].value_counts(normalize=True)

 1    0.811185
-1    0.188815
Name: safe_loans, dtype: float64

#### 这是一个不平衡数据， 好的贷款远比坏的贷款要多. 

## 选取用于预测的特征

In [6]:
cols = ['grade', 'term','home_ownership', 'emp_length']
target = 'safe_loans'

data = data[cols + [target]]
data.head()

,grade,term,home_ownership,emp_length,safe_loans
0,B,36 months,RENT,10+ years,1
1,C,60 months,RENT,< 1 year,-1
2,C,36 months,RENT,10+ years,1
3,C,36 months,RENT,10+ years,1
4,A,36 months,RENT,3 years,1


## 创建更为平衡的数据集  

* 对占多数的标签进行下采样  
* 注意有很多方法处理不平衡数据，下采样只是其中之一

In [7]:
data['safe_loans'].value_counts()


 1    99457
-1    23150
Name: safe_loans, dtype: int64

In [8]:

# use the percentage of bad and good loans to undersample the safe loans.
bad_ones = data[data[target] == -1]
safe_ones = data[data[target] == 1]
percentage = len(bad_ones)/float(len(safe_ones))

risky_loans = bad_ones
safe_loans = safe_ones.sample(frac=percentage, random_state=33)

# combine two kinds of loans
data_set = pd.concat([risky_loans, safe_loans], axis=0)

Now, let's verify that the resulting percentage of safe and risky loans are each nearly 50%.

In [9]:
data_set[target].value_counts(normalize=True)

-1    0.5
 1    0.5
Name: safe_loans, dtype: float64

## Preprocessing your features

In [10]:
def dummies(data, columns=['pclass','name_title','embarked', 'sex']):
    for col in columns:
        data[col] = data[col].apply(lambda x: str(x))
        new_cols = [col + '_' + i for i in data[col].unique()]
        data = pd.concat([data, pd.get_dummies(data[col], prefix=col)[new_cols]], axis=1)
        del data[col]
    return data

In [11]:
#grade, home_ownership, target
cols = ['grade', 'term','home_ownership', 'emp_length']
data_set = dummies(data_set, columns=cols)
data_set.head()

,safe_loans,grade_C,grade_F,grade_B,grade_D,grade_A,grade_E,grade_G,term_ 60 months,term_ 36 months,...,emp_length_3 years,emp_length_10+ years,emp_length_1 year,emp_length_9 years,emp_length_2 years,emp_length_8 years,emp_length_7 years,emp_length_5 years,emp_length_nan,emp_length_6 years
1,-1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,-1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,-1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
10,-1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
12,-1,0,0,1,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0


## 将数据分成训练集和测试集

重要的事情说三遍!!  

**把你的爪子从TEST DATA上拿开!!**   
**把你的爪子从TEST DATA上拿开!!**  
**把你的爪子从TEST DATA上拿开!!**  


In [12]:
train_data, test_data = train_test_split(data_set, test_size=0.2, random_state=33)
trainX, trainY = train_data[train_data.columns[1:]], pd.DataFrame(train_data[target])
testX, testY = test_data[test_data.columns[1:]], pd.DataFrame(test_data[target])

## 建自己的决策树!  

任务：  
1 实现根据error来选择最佳划分特征的函数best_split()  
2 实现根据entropy来选择最佳特征的函数best_split_entropy()  
3 实现树节点的类TreeNode  
4 实现模型的类MyDecisionTree  

#### 任务1， 实现根据error来选择最佳划分特征的函数best_split()  
约定树的左边对应target == 0， 树的右边对应target == 1

In [13]:
def count_errors(labels_in_node):
    if len(labels_in_node) == 0:
        return 0
    
    positive_ones = labels_in_node.apply(lambda x: x==1).sum()
    negative_ones = labels_in_node.apply(lambda x: x==-1).sum()
    
    return min(positive_ones, negative_ones)


def best_split(data, features, target):
    # return the best feature
    best_feature = None
    best_error = 2.0 
    num_data_points = float(len(data))  

    for feature in features:
        
        # 左分支对应当前特征为0的数据点
        left_split = data[data[feature] == 0]
        
        # 右分支对应当前特征为1的数据点
        right_split = data[data[feature] == 1] 
        
        # 计算左边分支里犯了多少错
        left_misses = count_errors(left_split[target])            

        # 计算右边分支里犯了多少错
        right_misses = count_errors(right_split[target])
            
        # 计算当前划分之后的分类犯错率
        error = (left_misses + right_misses) * 1.0 / num_data_points

        # 更新应选特征和错误率，注意错误越低说明该特征越好
        if error < best_error:
            best_error = error
            best_feature = feature
    return best_feature

#### 任务2， 实现根据entropy来选择最佳特征的函数best_split_entropy()  


In [14]:
def entropy(labels_in_node):
    # 二分类问题: 0 or 1
    n = len(labels_in_node)
    s1 = (labels_in_node==1).sum()
    if s1 == 0 or s1 == n:
        return 0
    
    p1 = float(s1) / n
    p0 = 1 - p1
    return -p0 * np.log2(p0) - p1 * np.log2(p1)


def best_split_entropy(data, features, target):
    
    best_feature = None
    best_info_gain = float('-inf') 
    num_data_points = float(len(data))
    # 计算划分之前数据集的整体熵值
    entropy_original = entropy(data[target])

    for feature in features:
        
        # 左分支对应当前特征为0的数据点
        left_split = data[data[feature] == 0]
        
        # 右分支对应当前特征为1的数据点
        right_split = data[data[feature] == 1] 
        
        # 计算左边分支的熵值
        left_entropy = entropy(left_split[target])            

        # 计算右边分支的熵值
        right_entropy = entropy(right_split[target])
            
        # 计算左边分支与右分支熵值的加权和（数据集划分后的熵值）
        entropy_split = len(left_split) / num_data_points * left_entropy + len(right_split) / num_data_points * right_entropy
        
        # 计算划分前与划分后的熵值差得到信息增益
        info_gain = entropy_original - entropy_split

        # 更新最佳特征和对应的信息增益的值
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_feature = feature
    return best_feature
    

#### 任务3，实现树节点的类TreeNode，每个树节点应该包含如下信息:  

   3.1 is_leaf: True/False  表示当前节点是否为叶子节点  
   
   3.2 prediction: 当前节点做全民公投的预测结果
   
   3.3 left: 左子树  
   
   3.4 right: 右子树 
   
   3.5 split_feature: 当前节点用来划分数据时所采用的特征

In [15]:
class TreeNode:
    def __init__(self, is_leaf, prediction, split_feature):
        self.is_leaf = is_leaf
        self.prediction = prediction
        self.split_feature = split_feature
        self.left = None
        self.right = None
        

#### 任务4，实现模型的类MyDecisionTree， 实现如下主要函数  
  
  
   4.1 fit(): 模型在训练集上的学习  
   
   4.2 predict(): 模型在数据集上的预测
   
   4.3 score(): 模型在测试集上的得分   
   
   
   
   
   为了实现4.1 - 4.3的方法， 需要实现如下辅助函数  
   4.4 create_tree(): 创建一棵树  
   
   4.5 create_leaf(): 创建叶子节点  
   
   4.6 predict_single_data(): 模型预测单个数据  
   
   4.7 count_leaves(): 统计模型的叶子数

In [16]:
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
class MyDecisionTree(BaseEstimator):
    
    def __init__(self, max_depth, min_error):
        self.max_depth = max_depth
        self.min_error = min_error
    
    def fit(self, X, Y, data_weights = None):
        
        data_set = pd.concat([X, Y], axis=1)
        features = X.columns
        target = Y.columns[0]
        self.root_node = self.create_tree(data_set, features, 
                               target, current_depth = 0, max_depth = self.max_depth, min_error=self.min_error)
        
        
    def predict(self, X):
        prediction = X.apply(lambda row: self.predict_single_data(self.root_node, row), axis=1)
        return prediction
        
        
    def score(self, testX, testY):
        target = testY.columns[0]
        result = self.predict(testX)
        return accuracy_score(testY[target], result)
    
    
    def create_tree(self, data, features, target, current_depth = 0, max_depth = 10, min_error=0):
        """
        探索三种不同的终止划分数据集的条件  
  
        termination 1, 当错误率降到min_error以下, 终止划分并返回叶子节点  
        termination 2, 当特征都用完了, 终止划分并返回叶子节点  
        termination 3, 当树的深度等于最大max_depth时, 终止划分并返回叶子节点
        """
        
    
        # 拷贝以下可用特征
        remaining_features = features[:]

        target_values = data[target]

        # termination 1
        if count_errors(target_values) <= min_error:
            print("Termination 1 reached.")     
            return self.create_leaf(target_values)

        # termination 2
        if len(remaining_features) == 0:
            print("Termination 2 reached.")    
            return self.create_leaf(target_values)    

        # termination 3
        if current_depth >= max_depth: 
            print("Termination 3 reached.")
            return self.create_leaf(target_values)



        # 选出最佳当前划分特征
        #split_feature = best_split(data, features, target)   #根据正确率划分
        split_feature = best_split_entropy(data, features, target)  # 根据信息增益来划分

        # 选出最佳特征后，该特征为0的数据分到左边，该特征为1的数据分到右边
        left_split = data[data[split_feature] == 0]
        right_split = data[data[split_feature] == 1]

        # 剔除已经用过的特征
        remaining_features = remaining_features.drop(split_feature)
        print("Split on feature %s. (%s, %s)" % (split_feature, str(len(left_split)), str(len(right_split))))

        # 如果当前数据全部划分到了一边，直接创建叶子节点返回即可
        if len(left_split) == len(data):
            print("Perfect split!")
            return self.create_leaf(left_split[target])
        if len(right_split) == len(data):
            print("Perfect split!")
            return self.create_leaf(right_split[target])

        # 递归上面的步骤
        left_tree = self.create_tree(left_split, remaining_features, target, current_depth + 1, max_depth, min_error)        
        right_tree = self.create_tree(right_split,remaining_features,target, current_depth + 1, max_depth, min_error)


        #生成当前的树节点
        result_node = TreeNode(False, None, split_feature)
        result_node.left = left_tree
        result_node.right = right_tree
        return result_node    
    
    
    
    def create_leaf(self, target_values):
        # 用于创建叶子的函数

        # 初始化一个树节点
        leaf = TreeNode(True, None, None)

        # 统计当前数据集里标签为+1和-1的个数，较大的那个即为当前节点的预测结果
        num_positive_ones = len(target_values[target_values == +1])
        num_negative_ones = len(target_values[target_values == -1])

        if num_positive_ones > num_negative_ones:
            leaf.prediction = 1
        else:
            leaf.prediction = -1

        # 返回叶子        
        return leaf 
    
    
    
    def predict_single_data(self, tree, x, annotate = False):   
        # 如果已经是叶子节点直接返回叶子节点的预测结果
        if tree.is_leaf:
            if annotate: 
                print("leaf node, predicting %s" % tree.prediction)
            return tree.prediction 
        else:
            # 查询当前节点用来划分数据集的特征
            split_feature_value = x[tree.split_feature]

            if annotate: 
                print("Split on %s = %s" % (tree.split_feature, split_feature_value))
            if split_feature_value == 0:
                #如果数据在该特征上的值为0，交给左子树来预测
                return self.predict_single_data(tree.left, x, annotate)
            else:
                #如果数据在该特征上的值为0，交给右子树来预测
                return self.predict_single_data(tree.right, x, annotate)    
    
    def count_leaves(self):
        return self.count_leaves_helper(self.root_node)
    
    def count_leaves_helper(self, tree):
        if tree.is_leaf:
            return 1
        return self.count_leaves_helper(tree.left) + self.count_leaves_helper(tree.right)
    


In [17]:
#my_decesion_tree = create_tree(train_data, features, target, current_depth = 0, max_depth = 10)
m = MyDecisionTree(max_depth = 10, min_error = 1e-15)

In [18]:
m.fit(trainX, trainY)

Split on feature grade_A. (31776, 5264)
Split on feature grade_B. (21587, 10189)
Split on feature grade_C. (12308, 9279)
Split on feature grade_D. (5553, 6755)
Split on feature term_ 60 months. (1743, 3810)
Split on feature grade_E. (459, 1284)
Split on feature emp_length_10+ years. (358, 101)
Split on feature emp_length_6 years. (328, 30)
Split on feature home_ownership_OTHER. (325, 3)
Split on feature emp_length_4 years. (297, 28)
Termination 3 reached.
Termination 3 reached.
Termination 1 reached.
Split on feature home_ownership_MORTGAGE. (23, 7)
Split on feature home_ownership_OWN. (21, 2)
Termination 3 reached.
Termination 3 reached.
Termination 1 reached.
Split on feature grade_F. (25, 76)
Split on feature home_ownership_OWN. (21, 4)
Split on feature home_ownership_RENT. (13, 8)
Termination 3 reached.
Termination 3 reached.
Split on feature grade_G. (0, 4)
Perfect split!
Split on feature home_ownership_RENT. (47, 29)
Split on feature home_ownership_OWN. (39, 8)
Termination 3 reac

In [19]:
m.score(testX, testY)

0.6193304535637149

### 决策树复杂度的探讨

In [20]:
m.count_leaves()

185

#### 探索不同树深度对决策树的影响  

1 max_depth = 3  
2 max_depth = 7  
3 max_depth = 15


In [21]:
model_1 = MyDecisionTree(max_depth = 3, min_error = 1e-15)
model_2 = MyDecisionTree(max_depth = 7, min_error = 1e-15)
model_3 = MyDecisionTree(max_depth = 15, min_error = 1e-15)


In [22]:
model_1.fit(trainX, trainY)
model_2.fit(trainX, trainY)
model_3.fit(trainX, trainY)

Split on feature grade_A. (31776, 5264)
Split on feature grade_B. (21587, 10189)
Split on feature grade_C. (12308, 9279)
Termination 3 reached.
Termination 3 reached.
Split on feature term_ 60 months. (9134, 1055)
Termination 3 reached.
Termination 3 reached.
Split on feature emp_length_nan. (5037, 227)
Split on feature home_ownership_RENT. (3142, 1895)
Termination 3 reached.
Termination 3 reached.
Split on feature term_ 60 months. (220, 7)
Termination 3 reached.
Termination 3 reached.
Split on feature grade_A. (31776, 5264)
Split on feature grade_B. (21587, 10189)
Split on feature grade_C. (12308, 9279)
Split on feature grade_D. (5553, 6755)
Split on feature term_ 60 months. (1743, 3810)
Split on feature grade_E. (459, 1284)
Split on feature emp_length_10+ years. (358, 101)
Termination 3 reached.
Termination 3 reached.
Split on feature emp_length_nan. (1223, 61)
Termination 3 reached.
Termination 3 reached.
Split on feature home_ownership_MORTGAGE. (1919, 1891)
Split on feature emp_le

In [23]:
print("model_1 training accuracy :", model_1.score(trainX, trainY))
print("model_2 training accuracy :", model_2.score(trainX, trainY))
print("model_3 training accuracy :", model_3.score(trainX, trainY))

model_1 training accuracy : 0.6173326133909287
model_2 training accuracy : 0.6229481641468683
model_3 training accuracy : 0.6266198704103672


In [24]:
print("model_1 testing accuracy :", model_1.score(testX, testY))
print("model_2 testing accuracy :", model_2.score(testX, testY))
print("model_3 testing accuracy :", model_3.score(testX, testY))

model_1 testing accuracy : 0.6173866090712743
model_2 testing accuracy : 0.6206263498920086
model_3 testing accuracy : 0.6187904967602592


In [25]:
print("model_1 complexity is: ", model_1.count_leaves())
print("model_2 complexity is: ", model_2.count_leaves())
print("model_3 complexity is: ", model_3.count_leaves())

model_1 complexity is:  8
model_2 complexity is:  74
model_3 complexity is:  384
